# Ethics for NLP: Spring 2022
# Homework 4 Privacy



## 1. Data Overview and Baseline

A major problem with utilizing web data as a source for NLP applications is the increasing concern for privacy, e.g., such as microtargeting. This homework is aimed at developing a method to obfuscate demographic features, in this case (binary) gender and to investigate the trade-off between obfuscating an users identity and preserving useful information.

The given dataset consists of Reddit posts (`post_text`) which are annotated with the gender (`op_gender`) of the user and the corresponding subreddit (`subreddit`) category.

*  `subreddit_classifier.pickle` pretrained subreddit classifier
*  `gender_classifier.pickle` pretrained gender classifier
*  `test.csv` your primary test data
*  `male.txt` a list of words commonly used by men
*  `female.txt` a list of words commonly used by women
*  `background.csv` additional Reddit posts that you may optionally use for training an obfuscation model

In [ ]:
from sklearn.metrics import accuracy_score
from pandas.core.frame import DataFrame
from typing import List, Tuple
import pandas
import pickle
import random
import nltk
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
def get_preds(cache_name: str, test: List[str]) -> List[str]:
    loaded_model, dictionary, transpose, train_bow = pickle.load(open(cache_name, 'rb'))
    X_test = transpose(test, train_bow, dictionary)
    preds = loaded_model.predict(X_test)
    return preds

In [ ]:
def run_classifier(test_file: str) -> Tuple[float]:
    test_data = pandas.read_csv(test_file)

    cache_name = 'gender_classifier.pickle'
    test_preds = get_preds(cache_name, list(test_data["post_text"]))
    gold_test = list(test_data["op_gender"])
    gender_acc = accuracy_score(list(test_preds), gold_test)
    print("Gender classification accuracy", gender_acc)

    cache_name = 'subreddit_classifier.pickle'
    test_preds = get_preds(cache_name, list(test_data["post_text"]))
    gold_test = list(test_data["subreddit"])
    subreddit_acc = accuracy_score(list(test_preds), gold_test)
    print("Subreddit classification accuracy", subreddit_acc)
    return gender_acc, subreddit_acc

In [ ]:
gender_acc, subreddit_acc = run_classifier("test.csv")

assert gender_acc == 0.646
assert subreddit_acc == 0.832

**Default accuracy:**
*   `Gender    classification accuracy: 0.646`
*   `Subreddit classification accuracy: 0.832`

## 2. Obfuscation of the Test Dataset
### 2.1 Random Obfuscated Dataset  (4P)
First, run a random experiment, by randomly swapping gender-specific words that appear in posts with a word from the respective list of words of the opposite gender.

*  Write a function to read the female.txt and male.txt files
*  Tokenize the posts („post_text“) using NLTK (0.5p)
*  For each post, if written by a man („M“) and containing a token from the male.txt, replace that token with a random one from the female.txt (1p)
*  For each post, if written by a woman („W“) and containing a token from the female.txt, replace that token with a random one from the male.txt (1p)
*  Save the obfuscated version of the test.csv in a separate csv file (using pandas and makes sure to name them accordingly) (0.5p)
*  Run the given classifier again, report the accuracy and provide a brief commentary on the results compared to the baseline (1p)

In [ ]:
def read_data(file_name: str) -> List[str]:
    """
    
    add your code here

    """
    return []

In [ ]:
male_words = read_data("add/your/path")
female_words = read_data("add/your/path")

assert len(male_words) == 3000
assert len(male_words) == 3000

In [ ]:
def obfuscate_gender_randomly(male_words: List[str], female_words: List[str], dataset_file_name: str) -> DataFrame:
  """
  
  add your code here
  
  """
  return DataFrame()

In [ ]:
file_name = "add file name"

In [ ]:
random_replaced_test = obfuscate_gender_randomly(male_words=male_words, female_words=female_words, dataset_file_name="test.csv")
random_replaced_test.to_csv(file_name)

In [ ]:
random_replaced_test = pandas.read_csv(file_name)
assert len(random_replaced_test) == 500
assert random_replaced_test["subreddit"][0] == "funny"
assert random_replaced_test["subreddit"][-1:].item() == "relationships"

In [ ]:
gender_acc, subreddit_acc = run_classifier(file_name)

assert gender_acc <= 0.5
assert subreddit_acc >= 0.7

**Report accuracy:**
*   `Gender    classification accuracy: `
*   `Subreddit classification accuracy: `
*   `Your commentary: ` ...

### 2.2 Similarity Obfuscated Dataset (4P)
In a second approach, refine the swap method. Instead of randomly selecting a word, use a similarity metric.


*  Instead of the first method replace the tokens by semantically similar tokens from the other genders token list. For that you may choose any metric for identifying semantically similar words, but you have to justify your choice. (Recommend: using cosine distance between pre-trained word embeddings) (2p)
*  Save the obfuscated version of the test.csv in a separate CSV file (using pandas and makes sure to name them accordingly) (0.5p)
*  Run the given classifier again, report the accuracy and provide a brief commentary on the results (compared to the baseline and your other results) (1p)
*  The classifiers accuracy for predicting the gender should be below random guessing (50%) and for the subreddit prediction it should be above 80% (0.5p)

In [ ]:
def obfuscate_gender_by_similarity(male_words: List[str], female_words: List[str], dataset_file_name: str) -> DataFrame:
  """
  
  add your code here
  
  """
  return DataFrame()

In [ ]:
"""
 you may use gensim models for example word2vec-google-news-300
"""

In [ ]:
file_name = "add file name"

In [ ]:
similarity_replaced_test = obfuscate_gender_by_similarity(male_words=male_words, female_words=female_words, dataset_file_name="./test.csv")
similarity_replaced_test.to_csv(file_name)

In [ ]:
similarity_replaced_test = pandas.read_csv(file_name)
assert len(similarity_replaced_test) == 500
assert similarity_replaced_test["subreddit"][0] == "funny"
assert similarity_replaced_test["subreddit"][-1:].item() == "relationships"

In [ ]:
gender_acc, subreddit_acc = run_classifier(file_name)

assert gender_acc <= 0.5
assert subreddit_acc >= 0.8

**Report accuracy:**
*   `Gender    classification accuracy: `
*   `Subreddit classification accuracy: ` 
*   `Your commentary: ` ...

### 2.3 Your Own Obfuscated Dataset (4P)
With this last approach, you can experiment by yourself how to obfuscate the posts.

*  Some examples: What if you randomly decide whether or not to replace words instead of replacing every lexicon word? What if you only replace words that have semantically similar enough counterparts? What if you use different word embeddings? (2p)
*  Save the obfuscated version of the test.csv in a separate csv file (using pandas and makes sure to name them accordingly) (0.5p)
*  Describe your modifications and report the accuracy and provide a brief commentary on the results compared to the baseline and your other results (1.5p)

In [ ]:
def obfuscate_gender(dataset_file_name: str) -> DataFrame:
  """

    add your own implemntation, you may add more functions and arguments
    
  """
  return DataFrame()

In [ ]:
file_name = "add file name"

In [ ]:
your_test = obfuscate_gender(dataset_file_name="./test.csv")
your_test.to_csv(file_name)

In [ ]:
your_test = pandas.read_csv(file_name)
assert len(your_test) == 500
assert your_test["subreddit"][0] == "funny"
assert your_test["subreddit"][-1:].item() == "relationships"

In [ ]:
gender_acc, subreddit_acc = run_classifier(file_name)

assert gender_acc <= 0.5
assert subreddit_acc >= 0.6

**Report accuracy:**
*   `Gender    classification accuracy: `
*   `Subreddit classification accuracy: ` 
*   `Your commentary: ` ...

### 3 Advanced Obfuscated Model (5P)
Develop your own obfuscation model using the provided background.csv for training. Your ultimate goal should be to obfuscate text so that the classifier is unable to determine the gender of an user (no better than random guessing) without compromising the accuracy of the subreddit classification task. To train a model that is good at predicting subreddit classification, but bad at predicting gender. The key idea in this approach is to design a model that does not encode information about protected attributes (in this case, gender). In your report, include a description of your model and results.

*  Develop your own classifier (3p)
*  Use only posts from the subreddits „CasualConversation“ and „funny“ (min. 1000 posts for each gender per subreddit) (0.5p)
*  Use sklearn models (MLPClassifier, LogisticRegression, etc.)
*  Use 90% for training and 10% for testing (0.5p)
*  In your report, include a description of your model and report the accuracy on the unmodified train data (your baseline here) as well as the modified train data and provide a brief commentary on the results (1p)

In [ ]:
"""

add your code here

"""

**Report accuracy:**
* Baseline:
  * `Gender    classification accuracy: `
  * `Subreddit classification accuracy: `
* Your Model: 
  * `Gender    classification accuracy: `
  * `Subreddit classification accuracy: ` 
*   `Your commentary: ` ...

### 4 Ethical Implications (3P)
Discuss the ethical implications of obfuscation and privacy based on the concepts covered in the lecture. Provide answers to the following points:

1.   What are demographic features (name at least three) and explain shortly some of the privacy violation risks? (1p)
2.   Explain the cultural and social implications and their effects? In this context discuss the information privacy paradox. You may refer to a recent example like the COVID-19 pandemic.  (1.5p)
3.   Name a at least three privacy preserving countermeasures  (0.5p)

Your Answer: ...

1. ...
2. ...
3. ...
